In [1]:
import pickle as pkl
import numpy as np

In [2]:
input_dim = 320

In [3]:
with open('Train_'+str(input_dim)+'.pkl', 'rb') as tr:
    X_train = pkl.load(tr)
tr.close()
# X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train.shape

(87844, 320)

In [4]:
with open('Val_'+str(input_dim)+'.pkl', 'rb') as val:
    X_val = pkl.load(val)
val.close()
# X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_val.shape

(1000, 320)

In [5]:
with open('Test_'+str(input_dim)+'.pkl', 'rb') as test:
    X_test = pkl.load(test)
test.close()
# X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test.shape

(1000, 320)

In [6]:
with open('Train_labels.txt', 'r') as l:
    train_labels = l.read().split('\n')
l.close()
len(train_labels)

87844

In [7]:
with open('Val_labels.txt', 'r') as v:
    val_labels = v.read().split('\n')
v.close()
len(val_labels)

1000

In [8]:
with open('Test_labels.txt', 'r') as t:
    test_labels = t.read().split('\n')
t.close()
len(test_labels)

1000

In [9]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(train_labels)

LabelEncoder()

In [10]:
y_train_labels = enc.transform(train_labels)
y_train_labels.shape

(87844,)

In [11]:
y_val_labels = enc.transform(val_labels)
y_val_labels.shape

(1000,)

In [12]:
y_test_labels = enc.transform(test_labels)
y_test_labels.shape

(1000,)

In [94]:
import keras
from keras.regularizers import l1, l2
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import EarlyStopping

In [14]:
y_train = to_categorical(y_train_labels)
y_train.shape

(87844, 10)

In [15]:
y_val = to_categorical(y_val_labels)
y_val.shape

(1000, 10)

In [16]:
y_test = to_categorical(y_test_labels)
y_test.shape

(1000, 10)

In [17]:
from sklearn.utils.class_weight import compute_class_weight

In [18]:
# Instantiate the label encoder
le = LabelEncoder()

# Fit the label encoder to our label series
le.fit(list(y_train_labels))

# Create integer based labels Series
y_integers = le.transform(list(y_train_labels))

In [19]:
class_weights = compute_class_weight('balanced', np.unique(y_train_labels), y_train_labels)
class_weights_dict = dict(zip(le.transform(list(le.classes_)), class_weights))

In [81]:
# Parameters
batch_size = 512
epochs = 100
num_classes = 10

In [102]:
model = Sequential()
model.add(Dense(32, activation='relu', input_shape=(320,)))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

In [103]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
print(model.summary())

Model: "sequential_11"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 32)                10272     
_________________________________________________________________
dense_30 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_31 (Dense)             (None, 10)                330       
Total params: 11,658
Trainable params: 11,658
Non-trainable params: 0
_________________________________________________________________
None


In [104]:
earlystop = EarlyStopping(monitor='val_acc', min_delta = 0.0001, patience=5, verbose=1, mode='auto')
callbacks_list = [earlystop]

In [105]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list,
          validation_data=(X_val, y_val), class_weight = class_weights_dict, verbose=1) # 

Train on 87844 samples, validate on 1000 samples
Epoch 1/100
87844/87844 [==============================] - 1s 17us/step - loss: 1.8620 - acc: 0.4713 - val_loss: 1.3406 - val_acc: 0.5770
Epoch 2/100
87844/87844 [==============================] - 1s 11us/step - loss: 1.1941 - acc: 0.6906 - val_loss: 1.0664 - val_acc: 0.6600
Epoch 3/100
87844/87844 [==============================] - 1s 11us/step - loss: 1.0132 - acc: 0.7263 - val_loss: 0.9443 - val_acc: 0.7050
Epoch 4/100
87844/87844 [==============================] - 1s 11us/step - loss: 0.9143 - acc: 0.7454 - val_loss: 0.8682 - val_acc: 0.7300
Epoch 5/100
87844/87844 [==============================] - 1s 12us/step - loss: 0.8492 - acc: 0.7605 - val_loss: 0.8181 - val_acc: 0.7460
Epoch 6/100
87844/87844 [==============================] - 1s 11us/step - loss: 0.8052 - acc: 0.7689 - val_loss: 0.7853 - val_acc: 0.7450
Epoch 7/100
87844/87844 [==============================] - 1s 10us/step - loss: 0.7743 - acc: 0.7757 - val_loss: 0.7618 - v

In [106]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

Val loss: 0.6948439946174622
Val accuracy: 0.778


In [107]:
y_pred = model.predict_classes(X_val)
confusion_matrix(y_pred, y_val_labels)

array([[71,  4,  1,  2,  1,  0,  3,  3,  0,  1],
       [ 1, 72,  4,  1,  0,  1,  1,  0,  1,  4],
       [ 7,  5, 82,  1,  0,  1,  1,  3,  0,  2],
       [ 7,  6,  1, 81,  0,  0,  3,  3,  0,  1],
       [ 0,  2,  1,  1, 90,  1,  2,  1,  8, 10],
       [ 2,  0,  1,  2,  1, 86,  4,  1,  1,  4],
       [ 4,  1,  0,  4,  0,  3, 80,  6,  3,  3],
       [ 2,  2,  4,  5,  0,  1,  5, 77,  3,  1],
       [ 1,  6,  3,  1,  5,  2,  1,  3, 74,  9],
       [ 5,  2,  3,  2,  3,  5,  0,  3, 10, 65]], dtype=int64)

In [108]:
test_scores = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Test loss: 0.7439253239631652
Test accuracy: 0.771


In [109]:
y_pred = model.predict_classes(X_test)
confusion_matrix(y_pred, y_test_labels)

array([[82,  4,  0,  0,  2,  1,  3,  1,  2,  1],
       [ 1, 74,  2,  1,  0,  0,  0,  3,  6,  3],
       [ 1,  2, 76,  1,  1,  0,  1,  3,  0,  1],
       [ 3,  1,  2, 75,  0,  1,  0,  3,  0,  1],
       [ 2,  2,  1,  1, 83,  1,  2,  0, 12,  7],
       [ 1,  1,  4,  7,  2, 84,  3,  2,  0,  2],
       [ 6,  5,  3,  9,  1,  5, 85,  5,  0,  0],
       [ 1,  4,  7,  4,  0,  5,  4, 74,  5,  3],
       [ 1,  6,  3,  1,  8,  3,  0,  4, 64,  8],
       [ 2,  1,  2,  1,  3,  0,  2,  5, 11, 74]], dtype=int64)

In [110]:
print(classification_report(y_pred, y_test_labels))

              precision    recall  f1-score   support

           0       0.82      0.85      0.84        96
           1       0.74      0.82      0.78        90
           2       0.76      0.88      0.82        86
           3       0.75      0.87      0.81        86
           4       0.83      0.75      0.79       111
           5       0.84      0.79      0.82       106
           6       0.85      0.71      0.78       119
           7       0.74      0.69      0.71       107
           8       0.64      0.65      0.65        98
           9       0.74      0.73      0.74       101

    accuracy                           0.77      1000
   macro avg       0.77      0.78      0.77      1000
weighted avg       0.77      0.77      0.77      1000



In [ ]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot feedforward.png', show_shapes=True, show_layer_names=True)